# Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

In [ ]:
masechtot=[]
with open('/Users/ephraimmeiri/gitEtc/Rashi Thesis work/references subproject/input/שמות מסכתות.txt') as f:
    data = f.read()
    masechtot = data.split(', ')
    
masechtot_bavli =[]
with open('/Users/ephraimmeiri/gitEtc/Rashi Thesis work/references subproject/input/שמות מסכתות בבלי.txt') as f:
    data = f.read()
    masechtot_bavli = data.split(', ')

perek_names = []
with open('/Users/ephraimmeiri/gitEtc/Rashi Thesis work/references subproject/input/perakim_names1.txt') as f:
    data = f.read()
    perek_names = data.split(', ')
perek_names2= []
for name in perek_names:
    if len(name.split())>1:     
        if name.split()[0] == "רבי":
            perek_names.append("ר' ".join(name.split()[1:]))
    if "\"" in name:
        perek_names2.append(name.replace("\"","\\\""))
for name in perek_names2:
    perek_names.append(name)
perek_names_bavli = []
with open('/Users/ephraimmeiri/gitEtc/Rashi Thesis work/references subproject/input/perakim_names_Bavli.txt') as f:
    data = f.read()
    perek_names_bavli = data.split(', ')
    
pirkei_mishna=[]
with open("/Users/ephraimmeiri/gitEtc/Rashi Thesis work/references subproject/input/perakim_names_MishnaOnly.txt",'r') as f:
    data = f.read()
    pirkei_mishna = data.split(', ')
    
numbers = ["קמא","אחרון","בתרא","עשירי","תשיעי","שמיני","שביעי","ששי","חמישי","דביעי","שלישי","שני","ראשון"]
for letter in ["י","ט","ח","ז","ו","ה","ד","ג","ב","א"]:
        numbers.append(letter+"'")
        numbers.append("י\""+letter)
        numbers.append("כ\""+letter)
rashi_dir= '/Users/ephraimmeiri/gitEtc/Rashi Thesis work/Rashi plaintext/json filtered'

In [ ]:
filtered_masechtot = masechtot.copy() # Names that are likely to be false positives
filtered_masechtot.remove("כלים")
filtered_masechtot.remove("ידים")

In [ ]:
def get_rashi_file(masechet):
    masechet = masechet.replace("%20"," ")
    file = rashi_dir+"/Rashi on "+masechet+".json"
    if os.path.exists(file):
        with open(file) as f:
            data = json.load(f)
            return data
    else:
        return None

In [ ]:
rashi_masechtot = ['Arakhin', 'Bekhorot', 'Chullin', 'Keritot', 'Menachot', 'Temurah', 'Zevachim', 'Beitzah', 'Chagigah', 'Eruvin', 'Pesachim', 'Megillah', 'Rosh Hashanah', 'Shabbat', 'Sukkah', 'Yoma', 'Gittin', 'Ketubot', 'Kiddushin', 'Sotah', 'Yevamot', 'Bava Batra', 'Bava Kamma', 'Bava Metzia', 'Avodah Zarah', 'Makkot', 'Sanhedrin', 'Shevuot', 'Niddah', 'Berakhot']

# Asking the questions

In [ ]:
def get_results(query):
    results = []
    for file in os.listdir(rashi_dir):
        if file == '.DS_Store.json':
            continue
        name = file.split(' - ')[0][9:-5]
        print(name,end='\t')
        with open(rashi_dir+"/"+file) as f:
            data = json.load(f)
            for daf,d in zip(data['text'],range(len(data['text']))):
                for line,l in zip(daf,range(len(daf))):
                    for comment,c in zip(line,range(len(line))):
                        cont = query(comment)
                        if cont:
                            results.append([name,d,c,cont,comment])
    return results

In [ ]:
def contains_masechet(text):
    results= None
    for masechet in masechtot_bavli:
        if "ב"+masechet in text:
            if results:
                results+=", "
                # print("Multi")
            else:
                results=""
            results+=masechet
    return results

# Right neighbors, for filtering

In [ ]:
def get_right_neighbors(query,dist):
    results = []
    for name in rashi_masechtot:
        file = get_rashi_file(name)
        if file:
            for d in range(len(file['text'])):
                for l in range(len(file['text'][d])):
                    for c in range(len(file['text'][d][l])):
                        cont = query(file['text'][d][l][c])
                        if cont:
                            for mas in cont.split(', '):
                                strs= file['text'][d][l][c].split(mas) 
                                words = file['text'][d][l][c].split()
                                words1= [strs[0].split(),strs[1].split()]
                                if len(words1[1])>1 and len(words1[0])>1 and (words[len(words)-1] != words1[1][0]) :
                                    if words1[1]:  # Check if words1[1] is not empty
                                        words1[1][0] = words1[0][-1].join(strs[1][0]) if words1[0] else ""  # Check if words1[0] is not empty before accessing its last element
                                    if words1[0]:  # Check if words1[0] is not empty
                                        words1[0] = words1[0][:-1]
                                if len(words1[0])>dist:
                                    results.append("".join(words1[0][-dist:]))
    # map to frequency
    res_set = set(results)
    freq = {}
    for res in res_set:
        freq[res] = results.count(res)
    freq = sorted(freq.items(), key=lambda x: x[1], reverse=True)
    return  freq

In [ ]:
mas_right_results = get_right_neighbors(contains_masechet,1)
print(len(mas_right_results))
mas_right_results

Ok not actually that useful, too many words and not enough context. We'll have to settle for this level of messiness

In [ ]:
mas_right_results = get_right_neighbors(contains_masechet,2)
print(len(mas_right_results))
mas_right_results

# Left neighbors -- do we have all the perek names?

In [ ]:
perek_names_one = []
for name in perek_names:
    if len(name.split())>1:     
        perek_names_one.append(name.split()[0])
        print(name.split()[0])
    else:
        perek_names_one.append(name)
        print("\t|"+name)

In [ ]:
def get_left_perek_neighbors(dist):
    results = []
    for name in rashi_masechtot:
        file = get_rashi_file(name)
        if file:
            for d in range(len(file['text'])):
                for l in range(len(file['text'][d])):
                    for c in range(len(file['text'][d][l])):
                        # cont = query(file['text'][d][l][c])
                        if "בפרק " in  file['text'][d][l][c]:
                            strs= file['text'][d][l][c].split("בפרק") 
                            words = file['text'][d][l][c].split()
                            words1= [strs[0].split(),strs[1].split()]
                            # if len(words1[1])>1 and len(words1[0])>1 and (words[len(words)-1] != words1[1][0]) :
                            #     if words1[1]:  # Check if words1[1] is not empty
                            #         words1[1][0] = words1[0][-1].join(strs[1][0]) if words1[0] else ""  # Check if words1[0] is not empty before accessing its last element
                            #     if words1[0]:  # Check if words1[0] is not empty
                            #         words1[0] = words1[0][:-1]
                            if len(words1[1])>dist:
                                neighbor = "".join(words1[1][:dist])
                                if neighbor not in perek_names_one and neighbor not in numbers:
                                    results.append("".join(words1[1][:dist]))
    # map to frequency
    res_set = set(results)
    freq = {}
    for res in res_set:
        freq[res] = results.count(res)
    freq = sorted(freq.items(), key=lambda x: x[1], reverse=True)
    return  freq

In [ ]:
perek_left_results = get_left_perek_neighbors(1)

In [ ]:
for name in perek_names:
    if "\"" in name:
        print(name)
print("----------------")
for name in perek_names_one:
    if "\"" in name:
        print(name)

In [ ]:
for name in perek_names:
    if "בנות" in name:
        print(name)
print("----------------")
for name in perek_names_one:
    if "בנות" in name:
        print(name)

In [ ]:
perek_left_results

Odd that some are showing up here, like ב"ש, אע"פ, and בנות כותים.

In [ ]:
def get_results(query):
    results = []
    for file in os.listdir(rashi_dir):
        if file == '.DS_Store.json':
            continue
        name = file.split(' - ')[0][9:-5]
        print(name,end='\t')
        with open(rashi_dir+"/"+file) as f:
            data = json.load(f)
            for daf,d in zip(data['text'],range(len(data['text']))):
                for line,l in zip(daf,range(len(daf))):
                    for comment,c in zip(line,range(len(line))):
                        cont = query(comment)
                        if cont:
                            results.append([name,d,c,cont,comment])
    return results
def contains_perek(text):
    for name in perek_names:
        if "בפרק "+name in text:
            return name

In [ ]:
perek_results = get_results(contains_perek)
perek_results_df = pd.DataFrame(perek_results,columns=["Masechet","Daf","Line","Perek","Comment"])

In [ ]:
perek_results_df.to_csv("perek_results1.csv")

# Defining the queries
Queries:
דמסכת
במסכת <שם
בפרק <שם
בפרק N
בפרק <?> דמסכת
במסכת <?> בפרק	

In [ ]:
def get_results(query):
    results = []
    for file in os.listdir(rashi_dir):
        if file == '.DS_Store.json':
            continue
        name = file.split(' - ')[0][9:-5]
        print(name,end='\t')
        with open(rashi_dir+"/"+file) as f:
            data = json.load(f)
            for daf,d in zip(data['text'],range(len(data['text']))):
                for line,l in zip(daf,range(len(daf))):
                    for comment,c in zip(line,range(len(line))):
                        cont = query(comment)
                        if cont:
                            results.append([name,d,c,cont,comment])
    return results


def contains_masechet(text):
    results= None
    for masechet in masechtot_bavli:
        if "ב"+masechet in text or "במסכת "+masechet in text or "ד"+masechet in text:
            if results:
                results+=", "
                # print("Multi")
            else:
                results=""
            results+=masechet
    return results

def contains_number(text):
    for num in numbers:
        if "בפרק "+num in text:
            return "בפרק "+num
    return None

def contains_perek(text):
    for name in perek_names:
        if "ב"+name in text or "בפרק "+name in text:
            return name

def contains_masechet_bPerek(text):
    results= None
    for masechet in masechtot_bavli:
        if masechet+" בפרק" in text:
            if results:
                results+=", "
            else:
                results=""    
            results+=masechet
        else:
            for name in perek_names:
                if masechet+"ב"+name in text:
                    if results:
                        results+=", "
                        # print("Multi")
                    else:
                        results=""
                    results+=  masechet+"ב"+name
    return results

def contains_perek_Dmasechet(text):
    results= None
    for masechet in masechtot_bavli:
        if "ד"+masechet in text and "פרק" in text:
            if results:
                results+=", "
                # print("Multi")
            else:
                results=""
            results+=masechet
    return results

In [56]:
masechet_results = get_results(contains_masechet)
masechet_bPerek_results = get_results(contains_masechet_bPerek)
masechet_Dperek_results = get_results(contains_perek_Dmasechet)

masechet_results_df = pd.DataFrame(masechet_results,columns=["Masechet","Daf","Line","References","Comment"])
masechet_bPerek_results_df = pd.DataFrame(masechet_bPerek_results,columns=["Masechet","Daf","Line","References","Comment"])
masechet_Dperek_results_df = pd.DataFrame(masechet_Dperek_results,columns=["Masechet","Daf","Line","References","Comment"])

Bava Kamma	Sanhedrin	Bava Batra	Avodah Zarah	Makkot	Yevamot	Kiddushin	Niddah	Pesachim	Menachot	Sukkah	Yoma	Sotah	Shevuot	Temurah	Gittin	Bekhorot	Berakhot	Arakhin	Chullin	Rosh Hashanah	Ketubot	Zevachim	Megillah	Beitzah	Shabbat	Eruvin	Bava Metzia	Chagigah	Keritot	Bava Kamma	Sanhedrin	Bava Batra	Avodah Zarah	Makkot	Yevamot	Kiddushin	Niddah	Pesachim	Menachot	Sukkah	Yoma	Sotah	Shevuot	Temurah	Gittin	Bekhorot	Berakhot	Arakhin	Chullin	Rosh Hashanah	Ketubot	Zevachim	Megillah	Beitzah	Shabbat	Eruvin	Bava Metzia	Chagigah	Keritot	Bava Kamma	Sanhedrin	Bava Batra	Avodah Zarah	Makkot	Yevamot	Kiddushin	Niddah	Pesachim	Menachot	Sukkah	Yoma	Sotah	Shevuot	Temurah	Gittin	Bekhorot	Berakhot	Arakhin	Chullin	Rosh Hashanah	Ketubot	Zevachim	Megillah	Beitzah	Shabbat	Eruvin	Bava Metzia	Chagigah	Keritot	

In [49]:
print(len(masechet_results))
# filter out double results (in perek/masechet but also in bperek_bmasechet or bperek_dmasechet))
text_set = set(masechet_bPerek_results_df['Comment'].unique())
text_set.update(masechet_Dperek_results_df['Comment'].unique())
masechet_results_df = masechet_results_df[~masechet_results_df.Comment.isin(text_set)]
print(len(masechet_results_df))

5292
4842


In [57]:
perek_results = get_results(contains_perek)
number_results = get_results(contains_number)
number_results_df = pd.DataFrame(number_results,columns=["Masechet","Daf","Line","References","Comment"])
perek_results_df = pd.DataFrame(perek_results,columns=["Masechet","Daf","Line","References","Comment"])

Bava Kamma	Sanhedrin	Bava Batra	Avodah Zarah	Makkot	Yevamot	Kiddushin	Niddah	Pesachim	Menachot	Sukkah	Yoma	Sotah	Shevuot	Temurah	Gittin	Bekhorot	Berakhot	Arakhin	Chullin	Rosh Hashanah	Ketubot	Zevachim	Megillah	Beitzah	Shabbat	Eruvin	Bava Metzia	Chagigah	Keritot	Bava Kamma	Sanhedrin	Bava Batra	Avodah Zarah	Makkot	Yevamot	Kiddushin	Niddah	Pesachim	Menachot	Sukkah	Yoma	Sotah	Shevuot	Temurah	Gittin	Bekhorot	Berakhot	Arakhin	Chullin	Rosh Hashanah	Ketubot	Zevachim	Megillah	Beitzah	Shabbat	Eruvin	Bava Metzia	Chagigah	Keritot	

In [ ]:
print(len(perek_results))
perek_results_df = perek_results_df[~perek_results_df.Comment.isin(text_set)]
print(len(perek_results_df))

In [46]:
print(len(number_results),len(number_results_df))
number_results_df = number_results_df[~number_results_df.Comment.isin(text_set)]
print(len(number_results_df))

345 179
Index(['Masechet', 'Daf', 'Line', 'Number', 'Comment'], dtype='object')
179
Masechet
Zevachim         18
Pesachim         15
Eruvin           14
Yevamot          12
Keritot          10
Ketubot           9
Gittin            9
Sanhedrin         9
Chullin           8
Yoma              7
Berakhot          7
Temurah           6
Menachot          5
Bekhorot          5
Kiddushin         5
Shevuot           5
Sotah             4
Bava Metzia       4
Avodah Zarah      4
Shabbat           3
Sukkah            3
Niddah            3
Arakhin           3
Bava Batra        2
Rosh Hashanah     2
Beitzah           2
Makkot            2
Bava Kamma        2
Megillah          1
Name: count, dtype: int64


# Results

In [ ]:
# Save output
masechet_results_df.to_csv("/Users/ephraimmeiri/gitEtc/Rashi Thesis work/references subproject/results/masechet_results.csv")
number_results_df.to_csv("/Users/ephraimmeiri/gitEtc/Rashi Thesis work/references subproject/results/number_results.csv")
perek_results_df.to_csv("/Users/ephraimmeiri/gitEtc/Rashi Thesis work/references subproject/results/perek_results.csv")
masechet_bPerek_results_df.to_csv("/Users/ephraimmeiri/gitEtc/Rashi Thesis work/references subproject/results/masechet_bPerek_results.csv")
masechet_Dperek_results_df.to_csv("/Users/ephraimmeiri/gitEtc/Rashi Thesis work/references subproject/results/masechet_Dperek_results.csv")

In [58]:
comparison_df = pd.concat([masechet_results_df['Masechet'].value_counts(),
                           perek_results_df['Masechet'].value_counts(),
                           number_results_df['Masechet'].value_counts(),
                           masechet_Dperek_results_df['Masechet'].value_counts(),
                           masechet_bPerek_results_df['Masechet'].value_counts()],
                           axis=1,sort=False,
                           keys=['Masechet','Perek','Number','Perek_Dmasechet','Masechet_bPerek'])
comparison_df['masechet/perek']= ((comparison_df['Masechet'])/
                                  (comparison_df['Perek']+comparison_df['Number']))

In [60]:
comparison_df

,Masechet,Perek,Number,Perek_Dmasechet,Masechet_bPerek,masechet/perek
Masechet,,,,,,
Shabbat,554,233,15,9.0,14,2.233871
Pesachim,383,234,31,18.0,29,1.445283
Chullin,337,236,26,15.0,26,1.286260
Zevachim,300,254,28,8.0,20,1.063830
Eruvin,264,152,25,11.0,15,1.491525
Sukkah,230,155,7,4.0,1,1.419753
Ketubot,229,115,12,2.0,15,1.803150
Kiddushin,215,79,7,3.0,7,2.500000
Menachot,207,132,10,7.0,24,1.457746
